# MEIRL

In [49]:
import pandas as pd
import numpy as np
import random

In [50]:
adjacency_matrix = pd.read_csv("adjacency_matrix.csv", index_col=0)

In [51]:
expert_feature_count = adjacency_matrix.copy()
expert_feature_count["feature_count"] = 0
expert_feature_count = expert_feature_count["feature_count"]
expert_feature_count = expert_feature_count.to_frame()
learner_feature_count = expert_feature_count.copy()
test_feature_count = expert_feature_count.copy()

In [52]:
# cross_validation
all_data = 40
training_number = int(all_data * 0.75)
test_number = int(all_data * 0.25)

# Import expert trajectory

### prepare dataset for training

In [53]:
for i in range (1,training_number):
    filename = "expert_trajectory/expert_trajectory " + str(i)
    file = open(filename, "r")
    expert_trajectory = file.read().split(', ')
    for j in expert_trajectory:
        current_count = int(expert_feature_count.loc[int(j), ["feature_count"]].values[0])
        add_count = expert_trajectory.count(j)
        new_count = current_count + add_count 
        expert_feature_count.loc[int(j), ["feature_count"]] = new_count
expert_feature_count = (expert_feature_count - expert_feature_count.min()) / expert_feature_count.max() - expert_feature_count.min()    

### prepare for testing

In [54]:
for i in range (training_number + 1 , all_data):
    filename = "expert_trajectory/expert_trajectory " + str(i)
    file = open(filename, "r")
    expert_trajectory = file.read().split(', ')
    for j in expert_trajectory:
        current_count = int(test_feature_count.loc[int(j), ["feature_count"]].values[0])
        add_count = expert_trajectory.count(j)
        new_count = current_count + add_count 
        test_feature_count.loc[int(j), ["feature_count"]] = new_count
test_feature_count = (test_feature_count - test_feature_count.min()) / test_feature_count.max() - test_feature_count.min()    

# Let's train

In [55]:
#Initialization of reward parameter
reward_parameter = expert_feature_count.copy()
reward_parameter.columns = ["reward_parameter"]

In [56]:
def update_position(current_position, epsilon):
    action_list = np.array(adjacency_matrix[adjacency_matrix[current_position] == 1].index)
    value_list = reward_parameter[reward_parameter.index.isin(action_list)]

    if np.random.random() > epsilon:
        updated_position = random.choice(value_list[value_list["reward_parameter"] == max(value_list["reward_parameter"])].index.tolist())
        gained_reward = value_list.loc[updated_position, "reward_parameter"]
    else:
        updated_position = random.choice(value_list["reward_parameter"].index.tolist())
        gained_reward = value_list.loc[updated_position, "reward_parameter"]

    return updated_position, gained_reward

In [57]:
for i in range (1,training_number):
    start_position = random.choice(adjacency_matrix.columns)
    current_position = start_position
    for i in range(50):
        current_position = str(update_position(current_position, 0.2)[0])
        learner_feature_count.loc[int(current_position), ["feature_count"]] += 1
learner_feature_count = (learner_feature_count - learner_feature_count.min()) / learner_feature_count.max() - learner_feature_count.min()    

# Compute the gradient 

In [60]:
gradient = expert_feature_count - learner_feature_count

In [ ]:
np.random.random() 

In [ ]:
for i in range (1,training_number):
    print(i)
    start_point = random.choice(adjacency_matrix.columns)
    goal_point = random.choice(adjacency_matrix.columns)

    current_point = start_point
    while (current_point != goal_point):
        current_point = str(update_position(current_point, 0.2)[0][0])
        